In [27]:
import os
from decimal import Decimal

os.environ["DJANGO_ALLOW_ASYNC_UNSAFE"] = "true"

from public_data.models import ZoneConstruite, Ocsge
from django.db.models import Subquery, OuterRef, DecimalField, CharField, F, Value, Sum, DecimalField
from django.db.models.functions import Cast
from django.contrib.gis.db.models.functions import Area, Transform, Intersection



In [2]:
zone = ZoneConstruite.objects.all().first()
zone

<ZoneConstruite: ZoneConstruite object (1)>

In [37]:
surface_built = Ocsge.objects.intersect(zone.mpoly).filter(couverture="CS1.1.1.1", year=zone.year)
surface_built = surface_built.aggregate(Sum("intersection_area"))
surface_built = surface_built['intersection_area__sum'].sq_m
surface_built = Decimal(surface_built)
surface_built

Decimal('3617.3503556320692950976081192493438720703125')

In [38]:
zone.surface

Decimal('20406.4850')

In [39]:
surface_built / zone.surface

Decimal('0.1772647447922593869104653800')

In [28]:
sub_qs = Ocsge.objects.filter(mpoly__intersects=OuterRef("mpoly"), couverture="CS1.1.1.1")
sub_qs = sub_qs.annotate(
    intersection=Intersection("mpoly", OuterRef("mpoly")),
    intersection_area=Area(Transform("intersection", 2154)),
)
sub_qs = sub_qs.filter().aggregate(Sum("intersection_area"))

AttributeError: 'ResolvedOuterRef' object has no attribute '_output_field_or_none'

In [33]:
ZoneConstruite.objects.all().annotate(
    surface_built=Subquery(
        Ocsge.objects.all().annotate(
            ext_mpoly=OuterRef("mpoly")
        ).filter(
            mpoly__intersects=F("ext_mpoly"), couverture="CS1.1.1.1"
        ).aggregate(
            Sum(Area(Transform(Intersection("mpoly", F("ext_mpoly")), 2154)))
        )[:1]
    )
).annotate(density=F("surface")/F("surface_built"))

AttributeError: 'ResolvedOuterRef' object has no attribute '_output_field_or_none'

In [41]:
del geom
geom = zone.mpoly.transform(2154)
geom.area

AttributeError: 'NoneType' object has no attribute 'area'

In [45]:
zone.mpoly.transform(2154, clone=True).area

20406.4849856344